In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
train = pd.read_csv("./data/train.csv", sep="###__###",encoding='utf-8',header = None)
test = pd.read_csv("./data/test.csv", sep="###__###",encoding='utf-8',header = None)

In [ ]:
train.columns = ['ID', 'Age', 'Gender', 'Education', 'Query_List']
train.head()

In [ ]:
train.info()

In [ ]:
train_isnull = train.loc[(train['Age'] == 0)|(train['Gender'] == 0)|(train['Education'] == 0),:]
print(train_isnull)

In [ ]:
import re
def remove_noise(document):
    noise_pattern = re.compile("|".join(["http\S+", ":http\S+", "ｗｗｗ.+?\t",'\d+\@\S+']))
    clean_text = re.sub(noise_pattern,"",document)
    return clean_text.strip()

In [ ]:
dataname = train.loc[0,'Query_List']
dataname

In [ ]:
remove_noise(dataname)

In [ ]:
train['Query_List'] = train['Query_List'].map(remove_noise)

In [ ]:
train.head()

In [ ]:
pip install jieba                                       

In [ ]:
import jieba

In [ ]:
str_test = '财经\thttp://pan.baidu.com/s/1plpjtn9\t捡手机被失主抢劫\thttps://yunpan.cn/ocsqfgtfya2ewj\t糖醋鲤鱼\tｗｗｗ．２０１６ｙｇ．ｃｏｍ'

In [ ]:
print(jieba.lcut(str_test))

In [ ]:
stop_words = 'C:/Users/lutianhao/Desktop/xiangmu/stopwords.txt'

In [ ]:
stopwords=pd.read_csv(stop_words,index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [ ]:
def preprocess_text(content_lines, sentences, category):
    out_f = open(target_path+"/"+category+".txt", 'w')
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = list(filter(lambda x:len(x)>1, segs)) #没有解析出来的新闻过滤掉
            segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
            sentences.append((" ".join(segs), category))
        except Exception as e:
            print(line)
            continue

In [ ]:
def process_df_text(content_lines):       
    segs=jieba.lcut(content_lines)
    segs = list(filter(lambda x:len(x)>1, segs)) #没有解析出来的新闻过滤掉
    segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
    return (" ".join(segs))

In [ ]:
train['Query_List'] = train['Query_List'].map(process_df_text)

In [ ]:
train.head()

In [ ]:
train['Age'].value_counts()

In [ ]:
Age_label = train[['Age','Query_List']]

In [ ]:
Age_label = Age_label[~(Age_label['Age'] == 0)]

In [ ]:
Age_label.head()

In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
X_age = Age_label['Query_List'].values.tolist()
Y_age = Age_label['Age'].values.tolist()

In [ ]:
vecl = CountVectorizer(ngram_range=(1, 1),  ### 词袋
                       lowercase=True, ### 转小写
                       binary=False,  ### 如果是True，tf的值只有0和1
                       min_df = 0.1, ###按比例或绝对数量,删除df超过max_df或者df小于min_df的word tokens;前提vocabulary=None
                       max_df = 0.9,
                       stop_words = None, ### list like ['the','an'....]
                       vocabulary=None,#### 自定义特征
                       max_features = 10000,  ###选择最大的max_features个特征
                       #token_pattern='\w{1,}',### regular expression,默认筛选长度大于等于2的字母和数字混合字符,参数analyzer设置为word时才有效
                       analyzer='word') ## 特征基于wordn-grams(词)还是character n-grams（字）) 

In [ ]:
def stratifiedkfold_cv(x, y,vec_word, clf,shuffle=True, n_folds=5):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    history_score=[]
    vec_word.fit(x)
    for train_index, test_index in stratifiedk_fold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train,y_test = y[train_index],y[test_index]
        clf.fit(vec_word.transform(X_train),y_train)
        history_score.append(clf.score(vec_word.transform(X_test), y_test)) 
    return history_score

In [ ]:
classifier = MultinomialNB()
history_score = stratifiedkfold_cv(np.array(X_age),np.array(Y_age),vecl,classifier)

In [ ]:
print(history_score)

In [ ]:
Gender_label = train[['Age','Query_List','Gender']]

In [ ]:
Gender_label = Gender_label[~(Gender_label['Gender'] == 0)]

In [ ]:
X_gender = Gender_label['Query_List'].values.tolist()
Y_gender = Gender_label['Gender'].values.tolist() 
Y_age = Gender_label['Age'].values.tolist()

In [ ]:
def stratifiedkfold_gender_cv(x, y,y_flag,vec_word, clf,shuffle=True, n_folds=5):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    history_score=[]
    vec_word.fit(x)
    for train_index, test_index in stratifiedk_fold.split(x, y_flag):
        X_train, X_test = x[train_index], x[test_index]
        y_train,y_test = y[train_index],y[test_index]
        clf.fit(vec_word.transform(X_train),y_train)
        history_score.append(clf.score(vec_word.transform(X_test), y_test)) 
    return history_score

In [ ]:
classifier_gender = MultinomialNB()

In [ ]:
history_score_gender = stratifiedkfold_gender_cv(np.array(X_gender),np.array(Y_gender),np.array(Y_age),vecl,classifier_gender)

In [ ]:
print(history_score_gender)

In [ ]:
Education_label = train[['Education','Query_List']]

In [ ]:
Education_label = Education_label[~(Education_label['Education'] == 0)]

In [ ]:
X_education = Education_label['Query_List'].values.tolist()
Y_education = Education_label['Education'].values.tolist()

In [ ]:
classifier_education = MultinomialNB()

In [ ]:
history_score_education = stratifiedkfold_cv(np.array(X_education),np.array(Y_education),vecl,classifier_education)

In [ ]:
print(history_score_education)

In [ ]:
print('age score  ',np.max(history_score))
print('gender score  ',np.max(history_score_gender))
print('education score  ',np.max(history_score_education))

In [ ]:
average_score = (np.max(history_score) + np.max(history_score_gender)+ np.max(history_score_education) )/3

In [ ]:
average_score